# FIT5148 - Distributed Databases and Big Data

# Group Assignment - Part A and B #

**Your Details:**
- Name: Roopak Thiyyathuparambil Jayachandran
- StudentID: 29567467
- Email: rthi0002@student.monash.edu

## Introduction 

StopFire is a campaign started by Monash University to predict and stop the fire in
Victorian cities. They have employed sensors in different cities of Victoria and have
collected a large amount of data. The data is so big that their techniques have failed to
provide the results on time to predict fire. They have hired us as the data analyst to
migrate their data to the NoSQL database (MongoDB). They want us to analyse their data
and provide them with results. In addition, they want us to build an application, a
complete setup from streaming to storing and analyzing the data for them using Apache
Kafka, Apache Spark Streaming and MongoDB.

In [4]:
check = open("climate_historic.csv", "r")

In [1]:
dic = {}
len(dic)

0

In [6]:
a = check.read()

In [9]:
li = a.split("\n")

In [12]:
li[1]

'948700,31/12/2016,19,56.8,7.9,11.1, 0.00I'

## Methodology ##
Assignment consists of 3 parts:
 * Identification of a suitable data model considering the given dataset
 * Data migration from structured dataset(csv) to NoSQL dataset(MongoDB). This part also involves solving of some questions related to the data
 * Create a standalone application involving data streaming, storing, analyzing and visualization.
 

### Given Dataset for data migration ###
 * hotspot_historic.csv
 * climate_historic.csv

### Exploring datasets ###
Hotspot historic : The hotspot historic dataset contains data related to demographics of a place and surface temperature along with confidence on a particular time. 

Total number of rows : 2668
Total number of columns : 6

On exploration it was found that the date range varies from 3rd May 2017 to 27th December 2017 with each day having multiple records. Also on further exploration it was found that there are 145 unique dates for which data has been recorded.

In [2]:
import pandas as pd

In [9]:
historic = pd.read_csv("hotspot_historic.csv")

In [10]:
historic.head()

,latitude,longitude,datetime,confidence,date,surface_temperature_celcius
0,-37.966,145.051,2017-12-27T04:16:51,78,27/12/2017,68
1,-35.541,143.311,2017-12-27T00:02:15,82,27/12/2017,63
2,-35.554,143.307,2017-12-27T00:02:15,67,27/12/2017,53
3,-35.543,143.316,2017-12-27T00:02:14,86,27/12/2017,67
4,-37.708,145.100,2017-12-25T04:29:08,80,25/12/2017,54


In [12]:
len(historic["date"].unique())

145

In [14]:
historic.dtypes

latitude                       float64
longitude                      float64
datetime                        object
confidence                       int64
date                            object
surface_temperature_celcius      int64
dtype: object

### Climate historic dataset : ###
The climate historic dataset contains data related to station, air temperature, relative humidity, windspeed knots, max wind speed and precipitation of a place on a particular date.
Total number of rows : 366
Total number of columns : 7

On exploration it was found that the date range varies from 31st December 2016 to 1st January 2018 with each day having single record.

In [15]:
climate = pd.read_csv("climate_historic.csv")

In [16]:
climate.shape

(366, 7)

In [20]:
climate.head()

,station,date,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation
0,948700,31/12/2016,19,56.8,7.9,11.1,0.00I
1,948700,2/01/2017,15,50.7,9.2,13.0,0.02G
2,948700,3/01/2017,16,53.6,8.1,15.0,0.00G
3,948700,4/01/2017,24,61.6,7.7,14.0,0.00I
4,948700,5/01/2017,24,62.3,7.0,13.0,0.00I


In [21]:
climate.dtypes

station                      int64
date                        object
air_temperature_celcius      int64
relative_humidity          float64
windspeed_knots            float64
max_wind_speed             float64
precipitation               object
dtype: object

## Data model proposal ##

After exploring both the dataset, it was identified that **date** is the common column between both the tables. And also each row in climate historic datset relates to multiple rows in hotspot historic dataset.
So the proposed data model BSON, will have climate historic as the outer dictionary, with a list of hotspot values as a value to one of the key named hotspots. Following is an example of a document from the proposed data model.


In [3]:
historic = pd.read_csv("hotspot_historic.csv")

In [8]:
len(historic["date"].unique())

145

In [16]:
a = historic[historic["date"] == "27/12/2017"]

In [18]:
a.head()

,latitude,longitude,datetime,confidence,date,surface_temperature_celcius
0,-37.966,145.051,2017-12-27T04:16:51,78,27/12/2017,68
1,-35.541,143.311,2017-12-27T00:02:15,82,27/12/2017,63
2,-35.554,143.307,2017-12-27T00:02:15,67,27/12/2017,53
3,-35.543,143.316,2017-12-27T00:02:14,86,27/12/2017,67


In [43]:
dic = {}
innerdic = {}
for index, row in climate.head().iterrows():
    li = []
    dic["date"] = row[1] 
    a = historic[historic["date"] == row[1]]
    print(type(a))
    for i, r in a.iterrows():
        innerdic["latitude"] = r[0]
        innerdic["longitude"] = r[1]
        li.append(innerdic)
    dic["his"] = li
    dic["station"] = row[0]

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [37]:
print(dic)

{'date': '31/12/2016', 'his': [], 'station': 948700}


In [20]:
import pandas as pd
cs = pd.read_csv("climate_streaming.csv")
aq = pd.read_csv("hotspot_AQUA_streaming.csv")
tr = pd.read_csv("hotspot_TERRA_streaming.csv")

In [18]:
cs.head(50)

,latitude,longitude,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation
0,-37.623,149.323,19,56.8,7.9,11.1,0.00I
1,-38.038,142.986,15,50.7,9.2,13.0,0.02G
2,-37.950,142.366,16,53.6,8.1,15.0,0.00G
3,-38.231,147.172,24,61.6,7.7,14.0,0.00I
4,-37.903,145.250,24,62.3,7.0,13.0,0.00I
5,-37.987,144.005,26,60.8,6.9,12.0,0.00I
6,-34.289,141.712,32,54.1,12.8,19.0,0.00I
7,-36.294,146.149,20,57.7,9.9,18.1,0.00I
8,-36.285,146.151,19,56.3,5.8,8.9,0.00I
9,-36.277,146.165,20,57.0,8.7,13.0,0.00I


In [13]:
cs.shape[0]

366

In [22]:
tr.head()

,latitude,longitude,confidence,surface_temperature_celcius
0,-37.966,145.051,78,68
1,-35.541,143.311,82,63
2,-35.554,143.307,67,53
3,-35.543,143.316,86,67
4,-37.708,145.100,80,54


In [27]:
tr[tr["latitude"] == -38.038]

,latitude,longitude,confidence,surface_temperature_celcius


In [1]:
a = [2,3,4]
print(a[-1])

4


In [2]:
import pandas as pd
columns = ["lat", "lon", "sur"]
df = pd.DataFrame(columns=columns)

In [3]:
df.head()

,lat,lon,sur


In [6]:
df = df.append(pd.Series([21, 21, 'Bangalore'], index=df.columns ), ignore_index=True)

In [7]:
df

,lat,lon,sur
0,21,21,Bangalore


In [14]:
df.iloc[0:0]

,lat,lon,sur


In [13]:
if 21 in df.lat.tolist():
    print("True")

True
